In [1]:
import pandas as pd
import numpy as np
import matplotlib . pyplot as plt

In [4]:
#1.2
df = pd.read_excel(r'C:/Users/Federico/Desktop/UdeSA/Maestría/Big Data/TP3/usu_hogar_T121.xlsx')
print (df)

                              CODUSU  ANO4  TRIMESTRE  NRO_HOGAR  REALIZADA  \
0      TQRMNOQUPHMKKUCDEOJAH00701956  2021          1          1          1   
1      TQRMNOPWXHMOKRCDEGLDF00701361  2021          1          1          1   
2      TQRMNOSUXHMMLPCDEGPDJ00701412  2021          1          1          1   
3      TQRMNORRQHKMKPCDEFOCD00653036  2021          1          1          1   
4      TQRMNOPYYHMKKRCDEHIBB00699594  2021          1          1          1   
...                              ...   ...        ...        ...        ...   
15418  TQRMNOSRSHJLKNCDEHNHB00686513  2021          1         72          1   
15419  TQRMNOPUPHMOKRCDEHNHB00698996  2021          1         51          1   
15420  TQRMNOPPTHKKKRCDEHPJB00655306  2021          1         71          1   
15421  TQRMNOSTVHLKSQCDEIJAH00651298  2021          1         51          1   
15422  TQRMNOPVYHKMLQCDEGGFB00653370  2021          1         51          1   

       REGION MAS_500  AGLOMERADO  PONDERA  IV1  ..

In [5]:
#1.2 continuación
df.drop(df[df['AGLOMERADO'] < 32.0].index, inplace=True)
df.drop(df[df['AGLOMERADO'] > 33.0].index, inplace=True)
print(df)

                              CODUSU  ANO4  TRIMESTRE  NRO_HOGAR  REALIZADA  \
15     TQRMNOQXUHJMMUCDEIJAH00693031  2021          1          1          1   
21     TQRMNOSRPHLLNRCDEIJAH00651171  2021          1          1          1   
24     TQRMNOPVUHKLMNCDEIIAD00655817  2021          1          1          1   
36     TQRMNORYTHLOPMCDEIJAH00655933  2021          1          1          1   
43     TQRMNOQPVHLMNNCDEIIAD00650542  2021          1          1          1   
...                              ...   ...        ...        ...        ...   
15373  TQRMNOQYSHMOLTCDEIJAH00700276  2021          1          1          1   
15382  TQRMNOURVHMNONCDEIIAD00700616  2021          1          1          1   
15391  TQRMNORRWHJMOPCDEIIAD00690119  2021          1          1          1   
15412  TQRMNOQWRHLOMPCDEIJAH00650932  2021          1          1          1   
15421  TQRMNOSTVHLKSQCDEIJAH00651298  2021          1         51          1   

       REGION MAS_500  AGLOMERADO  PONDERA  IV1  ..

In [6]:
df2 = pd.read_excel(r'C:/Users/Federico/Desktop/UdeSA/Maestría/Big Data/TP3/usu_individual_T121.xlsx')
df2.drop(df[df['AGLOMERADO'] < 32.0].index, inplace=True)
df2.drop(df[df['AGLOMERADO'] > 33.0].index, inplace=True)
print(df2)

                              CODUSU  ANO4  TRIMESTRE  NRO_HOGAR  COMPONENTE  \
0      TQRMNOPPQHLKLMCDEFIAH00646702  2021          1          1           1   
1      TQRMNOPPRHLKLMCDEFIAH00665104  2021          1          1           1   
2      TQRMNOPPRHMOLQCDEFIAH00701809  2021          1          1           1   
3      TQRMNOPPRHMOLQCDEFIAH00701809  2021          1          1           2   
4      TQRMNOPPRHMOLQCDEFIAH00701809  2021          1          1           3   
...                              ...   ...        ...        ...         ...   
46688  TQSMNOTRYHLOKPCDEOJAH00652060  2021          1          1           1   
46689  TQSMNOTRYHLOKPCDEOJAH00652060  2021          1          1           3   
46690  TQSMNOTRYHLOKPCDEOJAH00652060  2021          1          1           4   
46691  TQSMNOTRYHLOKPCDEOJAH00652060  2021          1          1           5   
46692  TQSMNOTRYHLOKPCDEOJAH00652060  2021          1          1           6   

       H15  REGION MAS_500  AGLOMERADO 

In [7]:
import pandas as pd
df3= pd.merge(df, df2, on = "CODUSU", how = "inner")
df3

,CODUSU,ANO4_x,TRIMESTRE_x,NRO_HOGAR_x,REALIZADA,REGION_x,MAS_500_x,AGLOMERADO_x,PONDERA_x,IV1,...,PDECIFR_y,ADECIFR_y,IPCF_y,DECCFR_y,IDECCFR_y,RDECCFR_y,GDECCFR_y,PDECCFR_y,ADECCFR_y,PONDIH_y
0,TQRMNOQXUHJMMUCDEIJAH00693031,2021,1,1,1,1,S,33,3255,1,...,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
1,TQRMNOQXUHJMMUCDEIJAH00693031,2021,1,1,1,1,S,33,3255,1,...,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
2,TQRMNOQXUHJMMUCDEIJAH00693031,2021,1,1,1,1,S,33,3255,1,...,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
3,TQRMNOQXUHJMMUCDEIJAH00693031,2021,1,1,1,1,S,33,3255,1,...,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
4,TQRMNOSRPHLLNRCDEIJAH00651171,2021,1,1,1,1,S,33,2331,1,...,0.0,6.0,17500.00,4.0,0.0,4.0,4.0,0.0,5.0,3953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,TQRMNOURVHMNONCDEIIAD00700616,2021,1,1,1,1,S,32,3772,2,...,0.0,4.0,58000.00,9.0,0.0,9.0,9.0,0.0,7.0,4325
4087,TQRMNORRWHJMOPCDEIIAD00690119,2021,1,1,1,1,S,32,1379,2,...,0.0,6.0,97500.00,10.0,0.0,10.0,10.0,0.0,9.0,1861
4088,TQRMNOQWRHLOMPCDEIJAH00650932,2021,1,1,1,1,S,33,8638,1,...,0.0,1.0,6333.33,1.0,0.0,1.0,1.0,0.0,1.0,11528
4089,TQRMNOQWRHLOMPCDEIJAH00650932,2021,1,1,1,1,S,33,8638,1,...,0.0,1.0,6333.33,1.0,0.0,1.0,1.0,0.0,1.0,11528


In [26]:
pd.set_option('display.max_columns', None) 
display(df3)

,CODUSU,ANO4_x,TRIMESTRE_x,NRO_HOGAR_x,REALIZADA,REGION_x,MAS_500_x,AGLOMERADO_x,PONDERA_x,IV1,IV1_ESP,IV2,IV3,IV3_ESP,IV4,IV5,IV6,IV7,IV7_ESP,IV8,IV9,IV10,IV11,IV12_1,IV12_2,IV12_3,II1,II2,II3,II3_1,II4_1,II4_2,II4_3,II5,II5_1,II6,II6_1,II7,II7_ESP,II8,II8_ESP,II9,V1,V2,V21,V22,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19_A,V19_B,IX_TOT,IX_MEN10,IX_MAYEQ10,ITF_x,DECIFR_x,IDECIFR_x,RDECIFR_x,GDECIFR_x,PDECIFR_x,ADECIFR_x,IPCF_x,DECCFR_x,IDECCFR_x,RDECCFR_x,GDECCFR_x,PDECCFR_x,ADECCFR_x,PONDIH_x,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,ANO4_y,TRIMESTRE_y,NRO_HOGAR_y,COMPONENTE,H15,REGION_y,MAS_500_y,AGLOMERADO_y,PONDERA_y,CH03,CH04,CH05,CH06,CH07,CH08,CH09,CH10,CH11,CH12,CH13,CH14,CH15,CH15_COD,CH16,CH16_COD,NIVEL_ED,ESTADO,CAT_OCUP,CAT_INAC,IMPUTA,PP02C1,PP02C2,PP02C3,PP02C4,PP02C5,PP02C6,PP02C7,PP02C8,PP02E,PP02H,PP02I,PP03C,PP03D,PP3E_TOT,PP3F_TOT,PP03G,PP03H,PP03I,PP03J,INTENSI,PP04A,PP04B_COD,PP04B1,PP04B2,PP04B3_MES,PP04B3_ANO,PP04B3_DIA,PP04C,PP04C99,PP04D_COD,PP04G,PP05B2_MES,PP05B2_ANO,PP05B2_DIA,PP05C_1,PP05C_2,PP05C_3,PP05E,PP05F,PP05H,PP06A,PP06C,PP06D,PP06E,PP06H,PP07A,PP07C,PP07D,PP07E,PP07F1,PP07F2,PP07F3,PP07F4,PP07F5,PP07G1,PP07G2,PP07G3,PP07G4,PP07G_59,PP07H,PP07I,PP07J,PP07K,PP08D1,PP08D4,PP08F1,PP08F2,PP08J1,PP08J2,PP08J3,PP09A,PP09A_ESP,PP09B,PP09C,PP09C_ESP,PP10A,PP10C,PP10D,PP10E,PP11A,PP11B_COD,PP11B1,PP11B2_MES,PP11B2_ANO,PP11B2_DIA,PP11C,PP11C99,PP11D_COD,PP11G_ANO,PP11G_MES,PP11G_DIA,PP11L,PP11L1,PP11M,PP11N,PP11O,PP11P,PP11Q,PP11R,PP11S,PP11T,P21,DECOCUR,IDECOCUR,RDECOCUR,GDECOCUR,PDECOCUR,ADECOCUR,PONDIIO,TOT_P12,P47T,DECINDR,IDECINDR,RDECINDR,GDECINDR,PDECINDR,ADECINDR,PONDII,V2_M,V3_M,V4_M,V5_M,V8_M,V9_M,V10_M,V11_M,V12_M,V18_M,V19_AM,V21_M,T_VI,ITF_y,DECIFR_y,IDECIFR_y,RDECIFR_y,GDECIFR_y,PDECIFR_y,ADECIFR_y,IPCF_y,DECCFR_y,IDECCFR_y,RDECCFR_y,GDECCFR_y,PDECCFR_y,ADECCFR_y,PONDIH_y
0,TQRMNOQXUHJMMUCDEIJAH00693031,2021,1,1,1,1,S,33,3255,1,NaN,9,1,NaN,2,1,1,2,NaN,1,1,1,2,2,2,2,9,3,1,1,1,1,1,2,0,2,0,1,NaN,1,NaN,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,4,0,4,55000.0,5.0,NaN,5.0,5.0,NaN,5.0,13750.00,3.0,NaN,3.0,3.0,NaN,4.0,5286,2,0,98,0,0,0,2021,1,1,1,1,1,S,33,3255,1,1,08/04/1963,57,2,1,1,2,0,7,2,5.0,1,NaN,1,NaN,5,1,3,0,NaN,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,60.0,0.0,1.0,1.0,2.0,2.0,3.0,2.0,2900.0,2.0,0.0,0.0,0.0,0.0,11.0,0.0,36202.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,5.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,55000.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55000,8,NaN,8,8.0,NaN,8,4298,0,55000.0,8,NaN,8,8.0,NaN,8,4458,0,0,0,0,0,0,0,0,0,0,0,0,0,55000.0,5.0,0.0,5.0,5.0,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
1,TQRMNOQXUHJMMUCDEIJAH00693031,2021,1,1,1,1,S,33,3255,1,NaN,9,1,NaN,2,1,1,2,NaN,1,1,1,2,2,2,2,9,3,1,1,1,1,1,2,0,2,0,1,NaN,1,NaN,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,4,0,4,55000.0,5.0,NaN,5.0,5.0,NaN,5.0,13750.00,3.0,NaN,3.0,3.0,NaN,4.0,5286,2,0,98,0,0,0,2021,1,1,2,1,1,S,33,3255,2,2,06/04/1971,49,2,1,1,2,0,6,2,2.0,1,NaN,1,NaN,5,3,0,4,NaN,0,0,0,0,0,0,0,0,0,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0.0,NaN,0,3255,0,0.0,0,NaN,0,0.0,NaN,0,3255,0,0,0,0,0,0,0,0,0,0,0,0,0,55000.0,5.0,0.0,5.0,5.0,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
2,TQRMNOQXUHJMMUCDEIJAH00693031,2021,1,1,1,1,S,33,3255,1,NaN,9,1,NaN,2,1,1,2,NaN,1,1,1,2,2,2,2,9,3,1,1,1,1,1,2,0,2,0,1,NaN,1,NaN,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,4,0,4,55000.0,5.0,NaN,5.0,5.0,NaN,5.0,13750.00,3.0,NaN,3.0,3.0,NaN,4.0,5286,2,0,98,0,0,0,2021,1,1,3,1,1,S,33,3255,3,2,22/11/2003,17,5,1,1,1,2,4,2,5.0,1,NaN,1,NaN,3,3,0,3,NaN,0,0,0,0

In [24]:
dfss = df3[['IV1','IV6','IV8','V5','ITF_y','IPCF_y']].copy()
dfss.describe(include='all')

,IV1,IV6,IV8,V5,ITF_y,IPCF_y
count,4091.000000,4091.00000,4091.000000,4091.000000,4091.000000,4091.000000
mean,1.311171,1.01320,1.001711,1.854070,54525.938890,18794.249553
std,0.489219,0.11835,0.041335,0.609196,77080.708389,28333.916280
min,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000
25%,1.000000,1.00000,1.000000,2.000000,0.000000,0.000000
50%,1.000000,1.00000,1.000000,2.000000,32200.000000,10000.000000
75%,2.000000,1.00000,1.000000,2.000000,77000.000000,26250.000000
max,4.000000,3.00000,2.000000,9.000000,750000.000000,307500.000000


In [32]:
dfprueba = df3.apply (pd.to_numeric, errors='coerce')
dfprueba.dropna(axis="columns", inplace=True)
dfprueba.T.drop_duplicates().T
dfprueba

,ANO4_x,TRIMESTRE_x,NRO_HOGAR_x,REALIZADA,REGION_x,AGLOMERADO_x,PONDERA_x,IV1,IV2,IV3,IV4,IV5,IV6,IV7,IV8,IV9,IV10,IV11,IV12_1,IV12_2,IV12_3,II1,II2,II3,II3_1,II4_1,II4_2,II4_3,II5,II5_1,II6,II6_1,II7,II8,II9,V1,V2,V21,V22,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19_A,V19_B,IX_TOT,IX_MEN10,IX_MAYEQ10,ITF_x,DECIFR_x,RDECIFR_x,GDECIFR_x,ADECIFR_x,IPCF_x,DECCFR_x,RDECCFR_x,GDECCFR_x,ADECCFR_x,PONDIH_x,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,ANO4_y,TRIMESTRE_y,NRO_HOGAR_y,COMPONENTE,H15,REGION_y,AGLOMERADO_y,PONDERA_y,CH03,CH04,CH06,CH07,CH08,CH09,CH10,CH11,CH12,CH13,CH15,CH16,NIVEL_ED,ESTADO,CAT_OCUP,CAT_INAC,PP02C1,PP02C2,PP02C3,PP02C4,PP02C5,PP02C6,PP02C7,PP02C8,PP02E,PP02H,PP02I,P21,DECOCUR,RDECOCUR,GDECOCUR,ADECOCUR,PONDIIO,TOT_P12,DECINDR,RDECINDR,GDECINDR,ADECINDR,PONDII,V2_M,V3_M,V4_M,V5_M,V8_M,V9_M,V10_M,V11_M,V12_M,V18_M,V19_AM,V21_M,T_VI,ITF_y,DECIFR_y,IDECIFR_y,RDECIFR_y,GDECIFR_y,PDECIFR_y,ADECIFR_y,IPCF_y,DECCFR_y,IDECCFR_y,RDECCFR_y,GDECCFR_y,PDECCFR_y,ADECCFR_y,PONDIH_y
0,2021,1,1,1,1,33,3255,1,9,1,2,1,1,2,1,1,1,2,2,2,2,9,3,1,1,1,1,1,2,0,2,0,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,4,0,4,55000.0,5.0,5.0,5.0,5.0,13750.00,3.0,3.0,3.0,4.0,5286,2,0,98,0,0,0,2021,1,1,1,1,1,33,3255,1,1,57,2,1,1,2,0,7,2,1,1,5,1,3,0,0,0,0,0,0,0,0,0,0,0,0,55000,8,8,8.0,8,4298,0,8,8,8.0,8,4458,0,0,0,0,0,0,0,0,0,0,0,0,0,55000.0,5.0,0.0,5.0,5.0,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
1,2021,1,1,1,1,33,3255,1,9,1,2,1,1,2,1,1,1,2,2,2,2,9,3,1,1,1,1,1,2,0,2,0,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,4,0,4,55000.0,5.0,5.0,5.0,5.0,13750.00,3.0,3.0,3.0,4.0,5286,2,0,98,0,0,0,2021,1,1,2,1,1,33,3255,2,2,49,2,1,1,2,0,6,2,1,1,5,3,0,4,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0.0,0,3255,0,0,0,0.0,0,3255,0,0,0,0,0,0,0,0,0,0,0,0,0,55000.0,5.0,0.0,5.0,5.0,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
2,2021,1,1,1,1,33,3255,1,9,1,2,1,1,2,1,1,1,2,2,2,2,9,3,1,1,1,1,1,2,0,2,0,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,4,0,4,55000.0,5.0,5.0,5.0,5.0,13750.00,3.0,3.0,3.0,4.0,5286,2,0,98,0,0,0,2021,1,1,3,1,1,33,3255,3,2,17,5,1,1,1,2,4,2,1,1,3,3,0,3,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0.0,0,3255,0,0,0,0.0,0,3255,0,0,0,0,0,0,0,0,0,0,0,0,0,55000.0,5.0,0.0,5.0,5.0,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
3,2021,1,1,1,1,33,3255,1,9,1,2,1,1,2,1,1,1,2,2,2,2,9,3,1,1,1,1,1,2,0,2,0,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,4,0,4,55000.0,5.0,5.0,5.0,5.0,13750.00,3.0,3.0,3.0,4.0,5286,2,0,98,0,0,0,2021,1,1,4,1,1,33,3255,3,2,12,5,1,1,1,2,4,2,1,1,3,3,0,3,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0.0,0,3255,0,0,0,0.0,0,3255,0,0,0,0,0,0,0,0,0,0,0,0,0,55000.0,5.0,0.0,5.0,5.0,0.0,5.0,13750.00,3.0,0.0,3.0,3.0,0.0,4.0,5286
4,2021,1,1,1,1,33,2331,1,9,1,3,1,1,1,1,1,1,1,2,2,2,9,3,2,0,1,1,1,2,0,2,0,1,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,4,0,4,70000.0,6.0,6.0,6.0,6.0,17500.00,4.0,4.0,4.0,5.0,3953,2,0,98,0,0,0,2021,1,1,1,1,1,33,2331,1,1,47,2,2,1,2,0,7,2,1,1,5,1,1,0,0,0,0,0,0,0,0,0,0,0,0,70000,9,9,9.0,9,3906,0,9,9,9.0,9,4416,0,0,0,0,0,0,0,0,0,0,0,0,0,70000.0,6.0,0.0,6.0,6.0,0.0,6.0,17500.00,4.0,0.0,4.0,4.0,0.0,5.0,3953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,2021,1,1,1,1,32,3772,2,1,1,9,1,1,1,1,1,1,1,2,2,2,1,1,2,0,2,2,2,0,0,0,0,3,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,1,0,1,58000.0,5.0,5.0,5.0,4.0,58000.00,9.0,9.0,9.0,7.0,4325,1,0,98,0,0,0,2021,1,1,1,1,1,32,3772,1,2,59,3,1,1,2,0,8,1,2,1,6,1,3,0,0,0,0,0,0,0,0,0,0,0,0,58000,9,8,8.0,6,4292,0,8,8,8.0,6,4547,0,0,0,0,0,0,0,0,0,0,0,0,0,580

In [25]:
def evalua_metodo (modelo, x_train, y_train, x_test, y_test):
    '''
    Esta función parte de un modelo predictivo definido previamente y luego evalúa sus principales métricas
    resultantes
    Imput:
          Modelo, x_train, y_train, x_test, y_test
    output:
          Matriz de confusión,AUC, ROC, ACCURACY SCORE
    '''
    accuracy_modelo = accuracy_score(y_test, y_pred_modelo)
    matriz_confusion = confusion_matrix(y_test, y_pred_modelo)
    auc = roc_auc_score(y_test, y_pred_modelo)
    return (accuracy_modelo, matriz_confusion, auc)